# EMGdecomPy Worflow

## Imports

In [23]:
from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.io import loadmat

## Load in data

In [4]:
x = loadmat('../data/raw/gl_10.mat')

## Run decomposition

In [6]:
output = decomposition(
    x["SIG"],
    discard=5,
    R=16,
    M=64,
    filter=True,
    lowcut=10,
    highcut = 900,
    fs=2048,
    order=6,
    peel=False,
    Tolx=10e-4,
    contrast_fun=skew,
    ortho_fun=gram_schmidt,
    max_iter_sep=10,
    l=31,
    sil_pnr=True,
    thresh=0.9,
    max_iter_ref=10,
    random_seed=None,
    verbose=False
)

Centred.
Extended.
Whitened.
Extracted source at iteration 4.
Extracted source at iteration 5.
Extracted source at iteration 11.
Extracted source at iteration 25.
Extracted source at iteration 27.
Extracted source at iteration 32.
Extracted source at iteration 35.
Extracted source at iteration 55.
Extracted source at iteration 59.
Extracted source at iteration 63.


## Visualize results

In [10]:
raw = x["SIG"]
signal = flatten_signal(x['SIG'])
signal = np.apply_along_axis(butter_bandpass_filter, axis=1, arr=signal, lowcut=10, highcut=900, fs=2048, order=8)
centered = center_matrix(signal)
c_sq = centered**2
c_sq_mean = c_sq.mean(axis=0)
pt = output['MUPulses']
mu_index = 0
shape_dict = muap_dict(raw, pt, l=31)

In [24]:
pt = output["MUPulses"]
shape_dict = muap_dict(raw, pt, l=31)

widget1 = widgets.Dropdown(
    options=np.arange(pt.shape[0]),
    value=0,
    description="Motor Unit:",
    disabled=False,
)

interact(dashboard, pt=fixed(pt), c_sq_mean=fixed(c_sq_mean), mu_index=widget1, raw=fixed(raw), shape_dict=fixed(shape_dict));

Traceback (most recent call last):
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/pyviz_comms/__init__.py", line 338, in _handle_msg
 self._on_msg(msg)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/panel/viewable.py", line 291, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/protocol/messages/patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/protocol/messages/patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/util/callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/bokeh/util/callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/panel/reactive.py", line 398, in _comm_event
 self._process_bokeh_event(event)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/panel/reactive.py", line 345, in _process_bokeh_event
 self._process_event(event)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/panel/pane/vega.py", line 237, in _process_event
 self.selection.param.update(**{name: value})
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/param/parameterized.py", line 1898, in update
 self_._batch_call_watchers()
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/param/parameterized.py", line 2059, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/Users/danielking/Library/Caches/pypoetry/virtualenvs/emgdecompy-gs61HjVg-py3.9/lib/python3.9/site-packages/param/paramet

Column
    [0] Vega(VConcatChart, debounce=10, selection=Selection)
    [1] ParamFunction(function)